In [72]:
# IMPORT PACKAGES
import pandas as pd
import time
import requests

In [65]:
def pullTrades(trade_url: str, params: dict) -> tuple:
    ''' Pull Kraken trades for specified start time.
    
    Args:
        trade_url (str): url of kraken rest api for trade history.
        params (dict): api call parameters.
    
    Returns:
        (tuple):
            - (list): trade lists of format [<price>, <volume>, <time>,
                      <buy/sell>, <market/limit>, <miscellaneous>, <trade_id>]
            - (str): unix time of last trade
    '''
    resp = requests.get(trade_url, params=params)
    assert(resp.status_code==200),"bad api call"
    result = resp.json()['result']
    return (result[params['pair']], result['last'])

In [83]:
# OBTAIN TRADE HISTORY
base_url   = 'https://api.kraken.com/0/public/'
trade_url  = base_url+'Trades'
start_time = 1451606400 # jan 1 2016 midnight gmt
end_time   = 1609372800 # dec 15 2022 midnight gmt

pair = 'XXBTZUSD'

trades = []

counter = 0
while start_time < end_time:
    # pull new trades
    params = {'pair': pair, 'since': str(start_time)}
    new_trades, start_time = pullTrades(trade_url, params)

    # adjust unix time
    start_time = int(int(start_time)/1e9)

    # update data
    trades  += new_trades
    counter += 1

    # space out calls
    time.sleep(0.3)

    #updates
    print(counter)
    print(time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(start_time)))

1
2016-01-11 23:41:38
2
2016-01-15 13:21:04
3
2016-01-16 21:51:36
4
2016-01-20 13:09:42
5
2016-01-21 14:00:27
6
2016-01-23 21:00:19
7
2016-01-25 18:11:01
8
2016-01-26 09:38:09
9
2016-01-27 23:35:38
10
2016-01-28 22:52:48
11
2016-01-29 10:17:17
12
2016-01-31 16:42:42
13
2016-02-02 01:42:21
14
2016-02-05 14:21:50
15
2016-02-07 18:31:19
16
2016-02-09 14:20:11
17
2016-02-10 05:13:51


In [4]:
'''

# clean up the data
df = pd.concat(token_dfs)
df = df[['date', 'cmc_id', 'usd_per_token', 'usd_volume_24h', 'usd_mcap']]
df['date'] = pd.to_datetime(df.date).dt.tz_localize(None)
df['date'] = df.date.dt.ceil('5min')
df = df.dropna()
df = df.drop_duplicates(subset=['date', 'cmc_id'])
dates_df = pd.concat((pd.DataFrame(data={'date': pd.date_range(first_date, last_date, freq='5min'),
                                        'cmc_id': 1}),
                      pd.DataFrame(data={'date': pd.date_range(first_date, last_date, freq='5min'),
                                        'cmc_id': 1027})))
df = df.merge(dates_df,
              on=['date', 'cmc_id'],
              how='outer',
              validate='one_to_one')
df = df.sort_values(by=['cmc_id', 'date'], ignore_index=True)
df = df.interpolate(method='ffill', limit_area='inside')
df = df.dropna()
assert(0==df.isnull().sum().sum())
df = df.sort_values(by=['date', 'cmc_id'], ignore_index=True)
'''

2017-12-31


NameError: name 'base_url' is not defined

In [ ]:
## SAVE DATA

df.to_csv('../3-data/raw/cmc_price_vol_mcap_btceth_5min.csv', index=False)
macro_df.to_csv('../3-data/raw/cmc_macro_timeseries_btceth_5min.csv', index=False)

,date,total_volume_24h,altcoin_market_cap,btc_dominance
0,2017-11-30T00:07:00.000Z,2.427597e+10,1.296200e+11,56.113804
1,2017-11-30T00:12:00.000Z,2.442784e+10,1.305865e+11,56.026665
2,2017-11-30T00:17:00.000Z,2.470249e+10,1.319481e+11,56.087906
3,2017-11-30T00:22:00.000Z,2.500412e+10,1.338523e+11,55.931969
4,2017-11-30T00:27:00.000Z,2.511601e+10,1.341620e+11,55.986496
...,...,...,...,...
7478,2022-08-25T23:20:00.000Z,6.434394e+10,6.288437e+11,39.616971
7479,2022-08-25T23:25:00.000Z,6.439341e+10,6.293038e+11,39.611672
7480,2022-08-25T23:30:00.000Z,6.431970e+10,6.298418e+11,39.618041
7481,2022-08-25T23:35:00.000Z,6.429181e+10,6.298992e+11,39.619576
